# Using SweetPea

This tutorial is based on the official documentation. For more information click [here](https://sweetpea-org.github.io/guide/usage.html#) to get to the official website.

So you’ve decided to design a factorial experiment. That’s excellent! Factorial experimental designs are a great way to build repeatable experiments with multiple independent variables. Let’s design our experiment in words first, and then build it in SweetPea.

## A Simple Stroop Experiment

For our example, we’ll be testing the [Stroop effect](https://en.wikipedia.org/wiki/Stroop_effect). From the Wikipedia article:

    In psychology, the Stroop effect is the delay in reaction time between congruent and incongruent stimuli.

One of the most well-known experiments to test the Stroop effect is to show a participant a series of words for colors that are also displayed in color. Sometimes, the color of the word is the same as the color in which the word is written — this is called congruence. Other times, the word and color are different, which is incongruence.

We have two apparent independent variables: the color and the text. We call independent variables factors in the realm of factorial design.

**Note:**
There is also a third factor: whether the pairing of the color and text is congruent or incongruent. This is a form of derived factor, and we’ll come back to it later.

To test this effect, we can construct a series of trials to administer to a participant, where each trial is a single color+text pairing. For this experiment, we will use the three colors red, green, and blue, and we will also use the names of those colors as the text. All together, this gives us 9 possible values for each trial:

||Red (Text)|Green (Text)|Blue (Text)|
|---|---|---|---|
|Red (Color)|<span style="color:#FF0000;">red</span>|<span style="color:#FF0000;">green</span>|<span style="color:#FF0000;">blue</span>|
|Green (Color)|<span style="color:#008000;">red</span>|<span style="color:#008000;">green</span>|<span style="color:#008000;">blue</span>|
|Blue (Color)|<span style="color:#0000FF;">red</span>|<span style="color:#0000FF;">green</span>|<span style="color:#0000FF;">blue</span>|

In the parlance of factorial design, these three colors constitute levels in each of the factors. That is to say that the ```color``` factor has three levels, and the ```text``` factor has three levels.

A trial will consist of showing one of the color and text pairs to a participant and asking them to identify the color in which the text is written. We will synthesize some trial sequences for our experiment now.

## Building the Simple Stroop Experiment

To build this simple Stroop experiment, we import and use the following SweetPea language forms: 
- ```Factor``` — constructs factors and their levels
- ```CrossBlock``` — combines the factors to produce trials
- ```synthesize_trials()``` — synthesizes trial sequences

To put it together, we do:

In [ ]:
from sweetpea import Factor, CrossBlock, synthesize_trials

# define the factors and their respective levels
text = Factor("text", ["red", "blue", "green"])
color = Factor("color", ["red", "blue", "green"])

# define the crossing of factors
block = CrossBlock([color, text], [color, text], [])

# synthesize the trials
experiments = synthesize_trials(block, 3)

The result of this synthesis is based on pseudo-random number generation, and so the output will not be the same every time. However, when we ran the code to write this tutorial, we saw the following output (your output should look similar, though probably not identical):

    1 trial sequences found.
    Experiment 0:
    color green | text blue
    color blue  | text green
    color green | text red
    color green | text green
    color red   | text green
    color red   | text blue
    color blue  | text blue
    color red   | text red
    color blue  | text red


In [ ]:
from sweetpea import print_experiments

print_experiments(block, experiments)

**Tip:**

The print_experiments() function is useful for printing the results of synthesis.

We generated a fully-crossed experiment: all possible color-text pairs were generated, though their order was randomized. 

We can see this by sorting a simplified representation of the experiment:

In [ ]:
from sweetpea import experiments_to_tuples

# We immediately access the first element of the returned list.
# This is because we only generated one trial run.
simple = experiments_to_tuples(block, experiments)[0]
for pair in sorted(simple):
    print(pair)

Because the color factor has $3$ levels and the text factor has $3$ levels, when we fully cross the factors we get $3*3=9$ resulting trials.

## SweetPea Feature Recap

In building our simple Stroop experiment, we used a few of the most important SweetPea forms. Let’s review them now.

### Simple Factors and Levels

Simple factors are factors that are composed only of simple levels. Simple levels are levels that are essentially just names and nothing more; they are not dependent on any other factors or levels.

While it is possible to import the `Level` class, it is usually not necessary (unless you want to assign weights to levels). Simple levels can only be put into simple factors, which in turn can only consist of simple levels, and we can create simple levels implicitly during `Factor` initialization.

When you construct a `Factor`, you also pass a list of levels to it. If those levels are not instances of the `Level` class, SweetPea will automatically convert them into instances of `Level`.

To put all this information together: you can create a simple factor composed of simple levels by just using the `Factor` initializer:

In [ ]:
from sweetpea import Factor

factor = Factor("factor_name", ("one", 2, 3.0, True))

print(f'The factor levels are: {len(factor.levels)}')
print(f'The factor name is: {factor["one"].name}')

We will discuss complex factors (also known as derived factors) a bit later.

### Block Creation

After you get your factors and levels together, you can create an experimental design `Block` using one of the appropriate functions. 

We showed how it looks to use `CrossBlock` in our simple example above. 

The function takes a number of arguments, but in the simplest case you need only do:

In [ ]:
from sweetpea import Factor, CrossBlock

# define the factors
f1 = Factor("f1", (1, 2, 3))
f2 = Factor("f2", ("a", "b", "c"))

# define the crossing of factors
block = CrossBlock(
    design = [f1, f2],
    crossing = [f1, f2], 
    constraints = [])

That is to say that when you’re only dealing with a simple experiment (an experiment comprised only of simple factors), you can probably just use a list of your factors as both the `design` and your `crossing`, and then hold the `constraints` empty with an empty list.

### Trial Synthesis

Once you have a complete experimental design in the form of a `Block`, you’re ready to use it to synthesize trials. 

Instead of creating only one trial, you can generate an arbitrary amount of trials by setting the `sampling` argument to the desired amount.

Regarding the sampling strategy, we used the default sampler in the above example, which is `IterateGen`. This automatically selects among strategies that implement non-replacement for a single request of multiple experiments, but the strategy may or may not provide uniformity for a single experiment. More sampling strategies can be found [here](https://sweetpea-org.github.io/api/sampling_strategies.html) in the official documentation.

In [ ]:
from random import Random
from sweetpea import synthesize_trials, IterateGen, RandomGen, print_experiments

# synthesize the trials using the IterateGen sampling strategy
experiments = synthesize_trials(
    block = block, 
    samples = 3,
    sampling_strategy = IterateGen
    )

print_experiments(block, experiments)

# synthesize the trials using the RandomGen sampling strategy
experiments = synthesize_trials(
    block = block, 
    samples = 3,
    sampling_strategy = RandomGen,
    )

print_experiments(block, experiments)

Which difference can you see between the synthesized trials using different types of sampling strategies?

### Working With Derived Levels

We’ve covered simple factors and levels, so now we move on to the more complex capabilities of SweetPea: derivations and constraints.

#### Derivation

Derivation is the process of creating new levels that depend in some way upon information contained in other levels from other factors — and sometimes other trials. In other words, derivation is what produces `DerivedLevels`.

Derivation is perhaps best explained through example. We resume the Stroop example from above, and return to the issue of congruency. Recall that we had produced two simple factors of three levels each. Now we would like to create a factor for `congruency` that has two levels: `congruent` and `incongruent`. A trial’s `congruency` is determined by the same trial’s `color` and `text`: if they align, then the `congruency` is `congruent`. Otherwise, the trial is `incongruent`.

Let’s create the `congruency` factor now. We start by recreating the `color` and `text` simple factors from before:

In [ ]:
from sweetpea import Factor

text = Factor("text", ["red", "blue", "green"])
color = Factor("color", ["red", "blue", "green"])

Next, we need to define the predicate functions that will be used to determine whether a color-text pair is congruent.

In [ ]:
def congruent(color: str, word: str) -> bool:
    return color == word

def incongruent(color: str, word: str) -> bool:
    return not congruent(color, word)
    # return color != word

Now, we can construct the derived levels. While simple levels can be constructed directly by the `Factor` during initialization, `DerivedLevel` instances must be manually instantiated. `DerivedLevels` also require a *derivation window* as an argument. We will discuss this more in-depth in a little bit, so for now just trust us that we want to use the `WithinTrial` for this particular job:

In [ ]:
from sweetpea import DerivedLevel, WithinTrial

con_level = DerivedLevel("congruent", WithinTrial(congruent, [color, text]))
inc_level = DerivedLevel("incongruent", WithinTrial(incongruent, [color, text]))

Finally, we can construct the `congruency` factor:

In [ ]:
congruency = Factor("congruency", [con_level, inc_level])

Now when we create a full crossing using `CrossBlock`, we will include the `congruency` factor with the rest of the design. 

However, it is not part of the crossing itself. The result of synthesizing trials from such a crossing will be a random arrangement of the following trials:

|Color|Text|Congruency|
|---|---|---|
|red|red|congruent|
|red|green|incongruent|
|red|blue|incongruent|
|green|red|incongruent|
|green|green|congruent|
|green|blue|incongruent|
|blue|red|incongruent|
|blue|green|incongruent|
|blue|blue|congruent|

We can verify this by using the `experiments_to_tuples()` function on the result of synthesizing one trial run from this design:

In [ ]:
from sweetpea import CrossBlock, synthesize_trials, experiments_to_tuples

design = [color, text, congruency]
crossing = [color, text]
block = CrossBlock(design, crossing, [])
experiments = synthesize_trials(block, 1)

for pair in sorted(experiments_to_tuples(block, experiments)[0]):
    print(pair)

#### Constraints

Sometimes when designing an experiment, you’d like to impose some constraints on the mechanisms that generate trial sequences. SweetPea has you covered.

Let’s say we look at the above list of trials and decide “Hmm, maybe we should ensure we don’t get too many `incongruent` trials in a row.” After all, there are six `incongruent` trials to just three `congruent` ones!

Arbitrarily, we will choose to limit trial sequences such that only two `incongruent` trials may appear in a row. This will be accomplished using the `AtMostKInARow()` function.

In [ ]:
# We resume from the previous session.

from sweetpea import AtMostKInARow

congruency_constraint = AtMostKInARow(2, congruency)
block = CrossBlock(design, crossing, [congruency_constraint])
experiments = synthesize_trials(block, 3, RandomGen)
print_experiments(block, experiments)

We can see from these outputs that we never get more than two trials in a row with the same `congruency` level selected. However, note that the constraint is not imposed across experiment boundaries: the final trial of the second experiment is `incongruent`, and the first two trials of the third experiment are also `incongruent`. This adds up to three consecutive trials! But this behavior is expected. The `AtMostKInARow` constraint only looks *within* a given experiment, not across experiments.

## Create your own Experimental Design for a Stroop Task

In the last sections you've learned about the most important features and concepts of SweetPea.

Let's deepen our understanding by writing our own experimental design for a Stroop Task.

We will introduce a third factor next to `color` and `text`. This factor is `task`.

The `task` factor denotes whether the participant should read the `text` or say the `color`.

Further, we will introduce a new derived factor, which will be `task_transition`.

This derived factor denotes whether the `task` was repeated or switched between two consecutive trials.
(Tip: You will need the `Transition` derivation. Read about it in the [official documentation](https://sweetpea-org.github.io/api.html)) 

#### Optional: 

In a next step, you can also think about adding `constraints` which handle event occurrences automatically.

`SweetPea` comes with more than only the presented `AtMostKInARow` method.

For more resources take a look at the [official documentation](https://sweetpea-org.github.io/api.html)

### Have fun implementing your own experimental design with SweetPea! 

In [ ]:
from sweetpea import Transition, AtLeastKInARow


# define the new factor task

# define the derived factor congruency and its levels congruent and incongruent
# 1. implement the methods congruent and incongruent
# 2. instantiate the DerivedLevel congruent_level and incongruent_level
# 3. insantiate the Factor congruency with the derived levels

# define the derived factor task_transition and its levels switch and repeat
# 1. implement the methods is_switch and is_repeat
# 2. instantiate the derived levels switch and repeat
# 3. instantiate the Factor task_transition with the derived levels

# define the new design without constraints for now
# 1. get all factors
# 2. get all crossings of non-derived factors
# 3. get the block and synthesize the trials
# 4. print the experiment
print('\nCreating experiment without constraints:\n')

# now we will add two constraints for each derived factor
# 1. at most two congruent trials in a row
# 2. at least three times the same task transition
print('\nCreating experiment with constraints:\n')

# now we will define one more constraint that will contradict the previous one:
# define a new constraint which clashes with one of the previously defined ones and see what happens
print('\nCreating experiment with contradicting constraints:\n')

In [ ]:
from sweetpea import Transition, AtLeastKInARow

# define the new factor task
task = Factor('task', ['read word', 'say color'])

# define the predict functions for the task transition
def is_switch(task):
    return task[0] != task[-1]

def is_repeat(task):
    return not is_switch(task)
    # return task[0] == task[-1]

# define the derived levels for the task factor
switch = DerivedLevel('switch', Transition(is_switch, [task]))
repeat = DerivedLevel('repeat', Transition(is_repeat, [task]))

# define the derived factor
task_transition = Factor('task transition', [switch, repeat])

In [ ]:
# define the new design
# first we are going to do that without any constraints
design = [color, text, congruency, task, task_transition]
crossing = [color, text, task]
block = CrossBlock(design, crossing, [])
experiments = synthesize_trials(block, 1)
print_experiments(block, experiments)

In [ ]:
# now we will add two constraints for each derived factor
constraint_congruency = AtMostKInARow(2, congruency)
constraint_transition = AtLeastKInARow(3, task_transition)
constraint_word = AtMostKInARow(3, text)
constraint_color = AtMostKInARow(3, color)
block = CrossBlock(design, crossing, [constraint_congruency, constraint_transition, constraint_word, constraint_color])
experiments = synthesize_trials(block, 1)
print_experiments(block, experiments)

In [ ]:
# now we will define one more constraint that will contradict the previous one
# the result is that the solver will overwrite the previous constraint
constraint_congruency_contra = AtLeastKInARow(3, congruency)
block = CrossBlock(design, crossing, [constraint_congruency, constraint_transition, constraint_congruency_contra])
experiments = synthesize_trials(block, 1)
print_experiments(block, experiments)